In [20]:
import pandas as pd
import numpy as np
import os
import ast
import re

# Read the prepared data file
prep_df = pd.read_csv("prep_movie_list.csv", sep=',', encoding='latin-1')
# prep_df.head(2)
#print(len(prep_df.index))

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
CV= CountVectorizer(max_features=5000, stop_words="english")
vector=CV.fit_transform(prep_df['listtags']).toarray()
#check random features name
# CV.get_feature_names()[3000]

In [17]:
tfidf=TfidfVectorizer(max_features=5000,analyzer='word',stop_words="english")
tfdf_features=tfidf.fit_transform(prep_df['listtags'])

from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(vector)
#prep_df[prep_df['title']=='DARK SKIES'].index[0]

sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x : x[1])[1:6] # key define second column basis sort
tfdf_similarity=cosine_similarity(tfdf_features)

In [18]:
#Recommender function
reco_list = []
def Recommend_Movies(movie):
    global reco_list
    try:
        movie_index=prep_df[prep_df['title']==movie].index[0]
        distance=similarity[movie_index]
        movies_list=sorted(list(enumerate(distance)), reverse=True, key=lambda x : x[1])[1:11]
        print(len(movies_list))
        if len(movies_list) > 0:
            for i in movies_list:
                #print(df8.iloc[i[0]].title)
                t=prep_df.iloc[i[0]].title
                #print(t)
                #print(t in reco_list)
                if t == movie or t in reco_list:
                    #continue
                    pass
                    #print('delete ' + t)
                else:
                    reco_list.append(t)
                    #print('add    ' + t)
        else:
            print('Movie '+ movie +' does not have any matching recommendation. Please try another one.')
    except:
        print('Movie '+ movie +' not present in Netflix. Please try another one.')

In [42]:
#User input section
'''
parameter 1:
    more than one word: movie title
'|' is the delimiter
parameter 2:
    single word: one genre of the movie
'|' is the delimiter
parameter 3:
    count of records to show > default 10
'''
errorKey = 0
inp_param = 'JAWS||'
param1 = str(inp_param.split('|',1)[0])
param2 = inp_param.split('|',1)[1].split('|',1)[0]
param3 = inp_param.rsplit('|',1)[1]

#print(param1)
#print(param2)
#print(param3)
reco_list = []
Recommend_Movies(param1)
#for x in reco_list:
#    print(x)
    
recolistdf = pd.DataFrame(reco_list)
recolistdf = recolistdf.rename(columns={0: 'title'})

#recolistdf = recolistdf.merge(q_movies[['title','demog_score']], on  = "title",how="left")
recolistdf = recolistdf.merge(prep_df[['title','cast','director','imdb_votes','keywords','genre','imdb_score','popularity_score','description','tags']], on  = "title",how="left")
recolistdf = recolistdf.drop_duplicates(subset = ['title'],keep = 'last').reset_index(drop = True)

recolistdf[['imdb_score']] = recolistdf[['imdb_score']].fillna(value=0)
recolistdf[['imdb_votes']] = recolistdf[['imdb_votes']].fillna(value=0)
recolistdf[['popularity_score']] = recolistdf[['popularity_score']].fillna(value=0)

#recolistdf['imdb_votes'] = recolistdf['imdb_votes'].astype(float)
recolistdf['imdb_score'] = recolistdf['imdb_score'].astype(float)
recolistdf['popularity_score'] = recolistdf['popularity_score'].astype(float)
recolistdf['composite_score'] = recolistdf['imdb_score']*0.6+recolistdf['popularity_score']*0.4
#recolistdf = recolistdf.drop(columns=['imdb_votes', 'imdb_score','popularity_score'])
recolistdf = recolistdf.sort_values(by=['composite_score','title'], ascending=False)

#recolistdf = recolistdf.loc[recolistdf['overview'].str.contains('fun',na=False)]
if len(param2) > 0:
    recolistdf = recolistdf.loc[recolistdf['genre'].str.contains(param2.lower())]
if len(param3) > 0:
    recolistdf = recolistdf.loc[recolistdf['cast'].str.lower().str.contains(param3.lower())]

recolistdf

10


,title,cast,director,imdb_votes,keywords,genre,imdb_score,popularity_score,description,tags,composite_score
3,THE CONJURING,"['Vera', 'Farmiga,', 'Patrick', 'Wilson,', 'Li...",James Wan,491048.0,['psychic vision demonology demonic ghost cruc...,"['', 'thriller', 'horror', 'thriller', 'horror...",7.5,171.680,"['When', 'a', 'family', 'starts', 'experiencin...","[['', 'thriller', 'horror', 'thriller', 'horro...",73.1720
4,PREY,"['David', 'Kross,', 'Hanno', 'Koffler,', 'Mari...",Thomas Sieben,7084.0,['bloody deaths bloody death stuck in car poac...,"['', 'drama', 'thriller', 'thriller', 'horror'...",4.6,76.039,"['A', 'hiking', 'trip', 'into', 'the', 'wild',...","[['', 'drama', 'thriller', 'thriller', 'horror...",33.1756
1,THE GREEN INFERNO,"['Ignacia', 'Allamand,', 'Daryl', 'Sabara,', '...",Eli Roth,43149.0,['amazon rainforest south america cannibal jun...,"['', 'horror', 'thriller', 'adventure', 'thril...",5.6,35.625,"['Determined', 'to', 'save', 'an', 'Amazon', '...","[['', 'horror', 'thriller', 'adventure', 'thri...",17.6100
0,JAWS 2,"['Roy', 'Scheider,', 'Lorraine', 'Gary,', 'Mur...",Jeannot Szwarc,0.0,['tourism water skiing high-tension current ki...,"['', 'thriller', 'horror', '', 'thriller', 'ho...",0.0,16.792,"['Four', 'years', 'after', 'the', 'last', 'dea...","[['', 'thriller', 'horror', '', 'thriller', 'h...",6.7168
2,JAWS: THE REVENGE,"['Lorraine', 'Gary,', 'Lance', 'Guest,', 'Mari...",Joseph Sargent,0.0,NaN,"['', 'thriller', 'adventure', '']",0.0,0.000,"['After', 'another', 'deadly', 'shark', 'attac...","[['', 'thriller', 'adventure', ''], ['After', ...",0.0000


In [45]:
netflix_titles = pd.read_csv("10k_titles.csv")
for movie in recolistdf['title']:
    print(movie.lower().title())
    if netflix_titles.loc[netflix_titles['title'] == movie.lower().title()]['overview'].empty:
        print("Title overview unavailable at this time.")
    else:
        print(re.sub("[^A-Za-z ]","",str(netflix_titles.loc[netflix_titles['title'] == movie.lower().title()]['overview'].values)))
    print('\n')

The Conjuring
Paranormal investigators Ed and Lorraine Warren work to help a family terrorized by a dark presence in their farmhouse Forced to confront a powerful entity the Warrens find themselves caught in the most terrifying case of their lives


Prey
When danger threatens her camp the fierce and highly skilled Comanche warrior Naru sets out to protect her people But the prey she stalks turns out to be a highly evolved alien predator with a technically advanced arsenal An American family on holiday in Africa becomes lost in a game reserve and stalked by vicious killer lions


The Green Inferno
A group of student activists travel from New York City to the Amazon to save the rainforest  However once they arrive in this vast green landscape they soon discover that they are not alone and that no good deed goes unpunished


Jaws 2
Police chief Brody must protect the citizens of Amity after a second monstrous shark begins terrorizing the waters


Jaws: The Revenge
Title overview unavailab